# Federated Xgboost

*The following codes are demos only. It's **NOT for production** due to system security concerns, please **DO NOT** use it directly in production.*

In this tutorial, we will learn how to use `SecretFlow` to train tree models for horizontal federation. Secretflow provides `tree modeling` capabilities for horizontal scenarios(`SFXgboost`), The usage of `SFXgboost` is similar to `XGBoost`, you can easily convert your existing xgboost program into a federated model for `SecretFlow`.

## Xgboost

XGBoost is an optimized distributed gradient boosting library designed to be highly efficient, flexible and portable. It implements machine learning algorithms under the Gradient Boosting framework

official tutorial [xgboost tutorials](https://xgboost.readthedocs.io/en/latest/tutorials/index.html)


### prepare secretflow devices 

In [1]:
%load_ext autoreload
%autoreload 2

import secretflow as sf
# In case you have a running secretflow runtime already.
sf.shutdown()

sf.init(['alice', 'bob', 'charlie'], num_cpus=8, log_to_driver=True)
alice, bob, charlie = sf.PYU('alice'), sf.PYU('bob'), sf.PYU('charlie')

2022-06-29 17:34:01.216674: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/rh/devtoolset-10/root/usr/lib64:/opt/rh/devtoolset-10/root/usr/lib:/opt/rh/devtoolset-10/root/usr/lib64/dyninst:/opt/rh/devtoolset-10/root/usr/lib/dyninst:/opt/rh/devtoolset-10/root/usr/lib64:/opt/rh/devtoolset-10/root/usr/lib
E0629 17:34:04.560745574  290369 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0629 17:34:04.590837393  290369 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0629 17:34:04.614931327  290369 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


### prepare dataset

In [2]:
from secretflow.data.simulation.dataset import load_dermatology_data

data_split = {
            alice: 0.5,
            bob: 1.0,
        }
file_uris = load_dermatology_data(party_ratio=data_split)

### xgboost example

In [3]:
import xgboost as xgb
import pandas as pd

df = pd.read_csv(file_uris[alice])
y = df['CLASS']
x = df.drop(columns="CLASS")
dtrain = xgb.DMatrix(x,y)
dtest = dtrain
params = {
            'max_depth': 4,
            'objective': 'multi:softmax',
            'min_child_weight': 1,
            'max_bin': 10,
            'num_class': 6,
            'eval_metric': 'merror',
        }
num_round = 4
watchlist = [(dtrain, 'train')]
bst = xgb.train(params, dtrain, num_round, evals=watchlist, early_stopping_rounds=2)


/home/xingmeng.zhxm/anaconda3/envs/secretflow/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
/home/xingmeng.zhxm/anaconda3/envs/secretflow/lib/python3.8/site-packages/xgboost/data.py:262: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[0]	train-merror:0.02326
[1]	train-merror:0.02326
[2]	train-merror:0.00000
[3]	train-merror:0.00000


### Then, How to do federated xgboost in secretflow?
1. Use federate Binning method based on iteration to calculate the global bucket information combined with the data of all sides, which was used as the candidate to enter the subsequent construction procedure
2. The data is input into each Client XGBoost engine to calculate G & H
3. Train federated boosting model  
   1) Data is reassigned to the node to be split  
   2) The sum of grad and the sum of hess are calculated according to the previously calculated binning buckets  
   3) Send the sum of grad and the sum of hess to server，server use secure aggregation to produce global summary，then choose best split point，Send best split info back to clients.  
   4) Clients Updates local model  
4. Finish training，and save model

Create 3 entities in the Secretflow environment [Alice, Bob, Charlie] Where 'Alice' and 'Bob' are clients, and `Charlie` is the server,then you can happily start `Federate Boosting`.

###  Prepare Data

In [4]:
from secretflow.data.horizontal import read_csv
from secretflow.security.aggregation.device_aggregator import DeviceAggregator
from secretflow.security.compare.plain_comparator import PlainComparator

hdf = read_csv(
    file_uris,
    aggregator=DeviceAggregator(charlie),
    comparator=PlainComparator(charlie),
)

(_run pid=2023708) 2022-06-29 17:34:15.796255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/rh/devtoolset-10/root/usr/lib64:/opt/rh/devtoolset-10/root/usr/lib:/opt/rh/devtoolset-10/root/usr/lib64/dyninst:/opt/rh/devtoolset-10/root/usr/lib/dyninst:/opt/rh/devtoolset-10/root/usr/lib64:/opt/rh/devtoolset-10/root/usr/lib
(_run pid=2023713) 2022-06-29 17:34:15.796258: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/rh/devtoolset-10/root/usr/lib64:/opt/rh/devtoolset-10/root/usr/lib:/opt/rh/devtoolset-10/root/usr/lib64/dyninst:/opt/rh/devtoolset-10/root/usr/lib/dyninst:/opt/rh/devtoolset-10/root/usr/lib64:/opt/rh/devtoolset-10/root/usr/lib


### Prepare Params

In [5]:
params = {# Xgboost parameter tutorial
         # https://xgboost.readthedocs.io/en/latest/parameter.html
         'max_depth': 4, # max depth
         'eta': 0.3, # learning rate
         'objective': 'multi:softmax', # objection function，support "binary:logistic","reg:logistic","multi:softmax","multi:softprob","reg:squarederror"
         'min_child_weight': 1, # The minimum value of weight
         'lambda': 0.1, # L2 regularization term on weights (xgb's lambda)
         'alpha': 0, # L1 regularization term on weights (xgb's alpha)
         'max_bin': 10, # Max num of binning
         'num_class':6, # Only required in multi-class classification
         'gamma': 0, # Same to min_impurity_split,The minimux gain for a split
         'subsample': 1.0, # Subsample rate by rows
         'colsample_bytree': 1.0, # Feature selection rate by tree
         'colsample_bylevel': 1.0, # Feature selection rate by level
         'eval_metric': 'merror',  # supported eval metric：                
                                    # 1. rmse 
                                    # 2. rmsle
                                    # 3. mape
                                    # 4. logloss
                                    # 5. error
                                    # 6. error@t
                                    # 7. merror
                                    # 8. mlogloss
                                    # 9. auc 
                                    # 10. aucpr 
         # Special params in SFXgboost
         # Required
         'hess_key': 'hess', # Required, Mark hess columns, optionally choosing a column name that is not in the data set
         'grad_key': 'grad', # Required，Mark grad columns, optionally choosing a column name that is not in the data set
         'label_key': 'CLASS', # Required，ark label columns, optionally choosing a column name that is not in the data set
}

### Create SFXgboost

In [6]:
from secretflow.ml.boost import SFXgboost

bst = SFXgboost(server=charlie, clients=[alice, bob])

run SFXgboost

In [7]:
bst.train(hdf, hdf, params=params, num_boost_round = 6)

(_run pid=2023708) 2022-06-29 17:34:20,775,775 WARNING [xla_bridge.py:backends:265] No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=2023713) 2022-06-29 17:34:20,836,836 WARNING [xla_bridge.py:backends:265] No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=2023706) 2022-06-29 17:34:21,000,0 WARNING [xla_bridge.py:backends:265] No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
(_run pid=2023706) 2022-06-29 17:34:26.103931: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/rh/devtoolset-10/root/usr/lib64:/opt/rh/devtoolset-10/root/usr/lib:/opt/rh/devtoolset-10/root/usr/lib64/dyninst:/opt/rh/devtoolset-10/root/usr/lib/dyninst:/opt/rh/devtoolset-10/root/usr/lib64:/opt/rh

(HomoBooster pid=2023706) [0]	train-merror:0.02326	valid-merror:0.02326
(HomoBooster pid=2023707) [0]	train-merror:0.02326	valid-merror:0.02326
(HomoBooster pid=2023711) [0]	train-merror:0.02326	valid-merror:0.02326
(HomoBooster pid=2023706) [1]	train-merror:0.03488	valid-merror:0.03488
(HomoBooster pid=2023707) [1]	train-merror:0.03488	valid-merror:0.03488
(HomoBooster pid=2023711) [1]	train-merror:0.03488	valid-merror:0.03488
(HomoBooster pid=2023706) [2]	train-merror:0.02326	valid-merror:0.02326
(HomoBooster pid=2023707) [2]	train-merror:0.02326	valid-merror:0.02326
(HomoBooster pid=2023711) [2]	train-merror:0.02326	valid-merror:0.02326
(HomoBooster pid=2023706) [3]	train-merror:0.02326	valid-merror:0.02326
(HomoBooster pid=2023707) [3]	train-merror:0.02326	valid-merror:0.02326
(HomoBooster pid=2023711) [3]	train-merror:0.02326	valid-merror:0.02326
(HomoBooster pid=2023706) [4]	train-merror:0.01163	valid-merror:0.01163
(HomoBooster pid=2023707) [4]	train-merror:0.01163	valid-merror:

Now our Federated XGBoost training is complete, where the BST is the federated Boost object

## Conclusion
* This tutorial introduces how to use tree models for training etc
* SFXgboost encapsulates the logic of the federated subtree model. Sfxgboost trained models remain compatible with XGBoost, and we can directly use the existing infrastructure for online prediction and so on.
* Next, you can try SFXgboost on your data, just need to follow this tutorial
